# Handling Field-of-View footprints with Multi-Order Coverage (v0)

This document explains how the Field-of-View (FoV) footprints can be easily and efficiently visualized and processed using [Multi-Order Coverage (MOC)](http://ivoa.net/documents/MOC/20140602/REC-MOC-1.0-20140602.pdf) maps based on [HEALPix](http://healpix.sourceforge.net/) sky tessellation. 

We convert the FoV footprints in a **MOC region** and subsequently, we show operations - union, intersection, difference and catalog queries -  by filtering the keywords reported in the FITS-header files.


### Testing version: Installation and initialization

Download the entire directory **FoV2moc** and run the tutorial there.

As the code runs, the results are displayed in real time in the [Aladin Sky Atlas](http://aladin.u-strasbg.fr/). Download Aladin from http://aladin.u-strasbg.fr/java/Aladin.jar and run it by typing

                             java -Xmx1024m -jar Aladin.jar
                             
We will need [healpy](https://healpy.readthedocs.org/en/latest/) for reading the probability sky map files, [mocpy](https://github.com/tboch/mocpy) for parsing and manipulating MOCs and [astropy](http://www.astropy.org/).  

**NOTE: download and install the modified version of mocpy**

    git clone https://github.com/ggreco77/mocpy
    cd mocpy
    (sudo) python3 setup.py install

In [ ]:
# testing - download the .py files in the folder
from aladinSAMP import AladinScriptCommands 
aladin = AladinScriptCommands()

from mocpy import MOC

# Coverage of a gravitational-wave skymap

Here we simulate a set of follow-up observations performed on a gravitational-wave sky localization by different instruments at various wavelenghts and observational times. There are 20 random observations in 6 different filters: 

| filter | n. footprints |  FoV size  |
|:------:|:-------------:|:----------:|
|   g    |       4       |  3°x3°     |
|   r    |       4       |  2°x2°     |
|   i    |       3       |  2.5°x2.5° |
|   z    |       3       |  1°x1°     |
|   J    |       3       |  1°x1°     |
|   H    |       3       |  1°x1°     |


### Loading the main EM-followUP observational parameters

For this tutorial we use the simulated sky maps from [**The First Two Years of Electromagnetic Follow-Up with Advanced LIGO and Virgo**](http://arxiv.org/abs/1404.5623) for compact binary Coalescence (CBC) sources. This is the event 450465.

In [ ]:
# get background
aladin.get_hips( "P/Mellinger/color" )

# loading the 10%/50%/90% MOC confidence levels of event id 450465
aladin.send_file("id_450465_moc01.fits")
aladin.rename("id_450465_moc01.fits")
aladin.set_moc("id_450465_moc01.fits")

aladin.send_file("id_450465_moc05.fits")
aladin.rename("id_450465_moc05.fits")
aladin.set_moc("id_450465_moc05.fits")

aladin.send_file("id_450465_moc09.fits")
aladin.rename("id_450465_moc09.fits")
aladin.set_moc("id_450465_moc09.fits")

In [ ]:
# testing - download the .py files in the folder
from fov2moc import Fov2Moc, group_headers

# fov id  
fov_ids = ["p1_g", "p2_g", "p3_g", "p4_g", 
           "p5_r","p6_r", "p7_r", "p8_r", 
           "p9_i", "p10_i", "p11_i", 
           "p12_z", "p13_z", "p14_z", 
           "p15_J","p16_J", "p17_J",
           "p18_H", "p19_H", "p20_H",]

# right ascension of the fov center in deg - ICRSd
ra_centers = [283.63515000, 272.97515, 271.2117, 274.48242, 
              280.68008, 275.07583, 275.07583, 273.07283, 
              273.1729, 267.1338, 272.15805,
              277.46756, 278.47405, 276.46107, 
              278.67837, 284.82329, 263.15765,
              285.52325, 284.51599, 284.51599,
              ]

# declination of the fov center in deg - ICRSd
dec_centers = [-02.36365, -05.96273, -04.98284, -07.18076, 
               -03.51348, -05.14232, -03.14232, -03.14232, 
               -04.38773, -05.1756, -08.1624, 
               -06.51501, -06.51501, -06.51501,
               -10.95555, -06.78932, -03.01335, 
               -06.88793, -06.88793, -05.88793,
               ]

# fov widths
fov_widths = [3, 3, 3, 3, 
              2, 2, 2, 2, 
              2.5, 2.5, 2.5, 
              1, 1, 1,
              1, 1, 1, 
              1, 1, 1, ]
             
# fov heights
fov_heights = [3, 3, 3, 3,
               2, 2, 2, 2, 
               2.5, 2.5, 2.5,
               1, 1, 1, 
               1, 1, 1,
               1, 1, 1, ]

# id instruments
instruments = ["instrument_g", "instrument_g","instrument_g","instrument_g",
               "instrument_r","instrument_r", "instrument_r","instrument_r",
               "instrument_i","instrument_i","instrument_i", 
               "instrument_z","instrument_z","instrument_z",
               "instrument_J","instrument_J", "instrument_J",
               "instrument_H","instrument_H","instrument_H",]

# id filters
filters = ["g", "g", "g", "g", 
           "r", "r", "r", "r", 
           "i", "i","i", 
           "z", "z", "z", 
           "J", "J", "J", 
           "H", "H", "H",]

# magnitude limits
lim_mags = [22, 22, 22, 22, 
            23, 23, 23, 23, 
            20, 20, 20, 
            18, 18, 18, 
            15, 15, 15,
            13, 13, 13,]

obs_times = ["2018-07-04 12:24:49.6","2018-07-04 12:24:49.6","2018-07-04 12:24:49.6","2018-07-04 12:24:49.6",
            "2018-07-04 12:24:49.6","2018-07-04 12:24:49.6","2018-07-04 12:24:49.6","2018-07-04 12:24:49.6",
            "2018-07-04 12:24:49.6","2018-07-04 12:24:49.6","2018-07-04 12:24:49.6",
            "2018-07-04 12:24:49.6","2018-07-04 12:24:49.6","2018-07-04 12:24:49.6",
            "2018-07-04 12:24:49.6","2018-07-04 12:24:49.6","2018-07-04 12:24:49.6",
            "2018-07-04 12:24:49.6","2018-07-04 12:24:49.6","2018-07-04 12:24:49.6",]

#zip  lists
zipped_list = zip(fov_ids, ra_centers, dec_centers, fov_widths, fov_heights, instruments, filters, 
                  lim_mags, obs_times)

# converting the FoV footprints into MOC - they are shown in the aladin folder named "pointings"
for fov_id, ra_center, dec_center, fov_width, fov_height, instrument, filter, lim_mag, obs_time in zipped_list:

    Fov2Moc(fov_id=fov_id, ra_center=ra_center, dec_center=dec_center,
            fov_width=fov_width, fov_height=fov_height, instrument=instrument,
            id_event='id_450465', pipeline='bayestar_F2Y',
            filter=filter, obs_time=obs_time, lim_mag=lim_mag )

<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/fov2moc/all_obs.jpg?raw=true" width="500" height="500"> 
**Aladin Screenshot - Field-of-View footprints -  20 random observations in 6 different filters.** 10%/50%/90% confidence levels are displayed in red, blue and orange.



<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/fov2moc/g_obs.jpg?raw=true" width="350" height="350" align = "left" > 
<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/fov2moc/r_obs.jpg?raw=true" width="350" height="350" align = "right"> 
<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/fov2moc/i_obs.jpg?raw=true" width="350" height="350" align = "left"> 
<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/fov2moc/z_obs.jpg?raw=true" width="350" height="350" align = "right"> 
<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/fov2moc/J_obs.jpg?raw=true" width="350" height="350" align = "left"> 
<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/fov2moc/H_obs.jpg?raw=true" width="350" height="350" align = "right"> 

**Aladin Screenshots** - *From left to right:* **g**, **r**, **i**, **z**, **J**  and **H** filters.

### Ex. 1 Group the Observation in g filter - measure the sky area and the skymap intersection (10% c.l.)

In [ ]:

# selecting the g-filter images
g_images = group_headers(fov_ids = fov_ids, filter="g")
print ("Images in g filter: ", g_images)

mocs=[]
for g_image in g_images:
    g_image = MOC.from_moc_fits_file(g_image)

    mocs.append(g_image)
    
# union the g-filter image in a bigger MOC
UNION_g = g_image.union_moc_list(mocs)

# writing the file "UNION_g"
UNION_g.write("UNION_g", write_to_file=True)

# sending to Aladin plan
aladin.send_file("UNION_g")
aladin.rename("UNION_g")


# square degrees in a whole sphere
from math import pi
square_degrees_sphere = (360.0**2)/pi

# printing area
area_sq2 = round( ( UNION_g.sky_fraction * square_degrees_sphere ), 1 )
print ( ' The union area is = ', area_sq2, 'sq. deg' )


# intersection between the g-filter images and the 90% c.l.
UNION_g = MOC.from_moc_fits_file("UNION_g")
cl_90 = MOC.from_moc_fits_file("id_450465_moc01.fits")

intersection_union_g_cl_90 = UNION_g.intersection(cl_90)

# printing area
area_sq2 = round( ( intersection_union_g_cl_90.sky_fraction * square_degrees_sphere ), 1 )
print ( ' The intersection between the all g-filter images and the 10% cl. is = ', area_sq2, 'sq. deg' )


# writing the intersection fie
intersection_union_g_cl_90.write("intersection_union_g_cl_10", write_to_file=True)

# sending to Aladin plan
aladin.send_file("intersection_union_g_cl_10")
aladin.rename("intersection_union_g_cl_10")



<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/fov2moc/inter_union_g_cl_10.jpg?raw=true" width="500" height="500"> 
**Aladin Screenshot -  in white the intersection between the g-filter images and the 10% cl.**


### Query the GLADE catalog inside the g-filter images

**In progress**

## Group the observation using the obs_time in the headers


**IN PROGRESS**